```
+--------------+--------------+------+-----+-------------------+-----------------------------------------------+
| Field        | Type         | Null | Key | Default           | Extra                                         |
+--------------+--------------+------+-----+-------------------+-----------------------------------------------+
| id           | int          | NO   | PRI | NULL              | auto_increment                                |
| name         | varchar(20)  | YES  |     | NULL              |                                               |
| datetime     | timestamp    | NO   |     | CURRENT_TIMESTAMP | DEFAULT_GENERATED                             |
| status       | varchar(10)  | YES  |     | NULL              |                                               |
| date_updated | timestamp    | NO   |     | CURRENT_TIMESTAMP | DEFAULT_GENERATED on update CURRENT_TIMESTAMP |
| filepath     | varchar(200) | YES  |     | NULL              |                                               |
+--------------+--------------+------+-----+-------------------+-----------------------------------------------+
```

In [1]:
from sqlalchemy import create_engine, orm, MetaData
engine = create_engine('mysql://root@localhost/sandbox', echo=False)
meta = MetaData(bind=engine)
meta.reflect()
class Excel(object):
    def __init__(self, name, status, filepath):
        self.name = name
        self.status = status
        self.filepath = filepath
        
    def __repr__(self):
        return f'Excel<{self.id}-{self.datetime}-{self.date_updated}:<{self.name}><{self.status}><{self.filepath}>'

orm.Mapper(Excel, meta.tables['excel_from_hell'])

<Mapper at 0x7f888c34a580; Excel>

## Session start

In [2]:
import pandas as pd

In [3]:
session = orm.Session(bind=engine)

In [4]:
data_for_table = {
    'name': ['30_Астрахань_20210215', '57_Орел_20210215', '58_Пенза_20210208'],
    'status': ['new', 'new', 'new'],
    'filepath': ['source_files/30_Астрахань_20210215.xls', 
                 'source_files/57_Орел_20210215.xls', 
                 'source_files/58_Пенза_20210208.xlsx']
}
df = pd.DataFrame(data=data_for_table)

In [5]:
df

,name,status,filepath
0,30_Астрахань_20210215,new,source_files/30_Астрахань_20210215.xls
1,57_Орел_20210215,new,source_files/57_Орел_20210215.xls
2,58_Пенза_20210208,new,source_files/58_Пенза_20210208.xlsx


## Convert to dict:

In [6]:
data_dict = df.to_dict(orient='records')

In [7]:
data_dict

[{'name': '30_Астрахань_20210215',
  'status': 'new',
  'filepath': 'source_files/30_Астрахань_20210215.xls'},
 {'name': '57_Орел_20210215',
  'status': 'new',
  'filepath': 'source_files/57_Орел_20210215.xls'},
 {'name': '58_Пенза_20210208',
  'status': 'new',
  'filepath': 'source_files/58_Пенза_20210208.xlsx'}]

In [8]:
for record in data_dict:
    print(record)

{'name': '30_Астрахань_20210215', 'status': 'new', 'filepath': 'source_files/30_Астрахань_20210215.xls'}
{'name': '57_Орел_20210215', 'status': 'new', 'filepath': 'source_files/57_Орел_20210215.xls'}
{'name': '58_Пенза_20210208', 'status': 'new', 'filepath': 'source_files/58_Пенза_20210208.xlsx'}


Add to base (FAKE)

In [9]:
for record in data_dict:
    display(Excel(name=record['name'], status=record['status'], filepath=record['filepath']))

Excel<None-None-None:<30_Астрахань_20210215><new><source_files/30_Астрахань_20210215.xls>

Excel<None-None-None:<57_Орел_20210215><new><source_files/57_Орел_20210215.xls>

Excel<None-None-None:<58_Пенза_20210208><new><source_files/58_Пенза_20210208.xlsx>

## Add to session:

In [10]:
for record in data_dict:
    session.add(Excel(name=record['name'], status=record['status'], filepath=record['filepath']))

Check for new records:

In [11]:
session.new

IdentitySet([Excel<None-None-None:<30_Астрахань_20210215><new><source_files/30_Астрахань_20210215.xls>, Excel<None-None-None:<57_Орел_20210215><new><source_files/57_Орел_20210215.xls>, Excel<None-None-None:<58_Пенза_20210208><new><source_files/58_Пенза_20210208.xlsx>])

In [12]:
session.commit()
session.close()

## Update names:

In [13]:
session = orm.Session(bind=engine)

New names list:

In [21]:
new_names = [('58_Penza_20210208', 'source_files/58_Penza_20210208.xlsx'),
('30_Astrakhan_20210215', 'source_files/30_Astrakhan_20210215.xls'),
('57_Orel_20210215', 'source_files/57_Orel_20210215.xls')]

In [28]:
session.query(Excel).all()

[Excel<1-2021-02-08 12:08:16-2021-02-08 12:08:16:<30_Астрахань_20210215><new><source_files/30_Астрахань_20210215.xls>,
 Excel<2-2021-02-08 12:08:16-2021-02-08 12:08:16:<57_Орел_20210215><new><source_files/57_Орел_20210215.xls>,
 Excel<3-2021-02-08 12:08:16-2021-02-08 12:08:16:<58_Пенза_20210208><new><source_files/58_Пенза_20210208.xlsx>]

In [29]:
for n, record in enumerate(session.query(Excel).all()):
    record.name = new_names[n][0]
    record.filepath = new_names[n][1]

In [30]:
session.query(Excel).all()

[Excel<1-2021-02-08 12:08:16-2021-02-08 12:08:16:<58_Penza_20210208><new><source_files/58_Penza_20210208.xlsx>,
 Excel<2-2021-02-08 12:08:16-2021-02-08 12:08:16:<30_Astrakhan_20210215><new><source_files/30_Astrakhan_20210215.xls>,
 Excel<3-2021-02-08 12:08:16-2021-02-08 12:08:16:<57_Orel_20210215><new><source_files/57_Orel_20210215.xls>]

In [32]:
session.commit()